In [1]:
import open3d as o3d
import numpy as np
from symlink import symbolic_dir

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## VAmos a cargar objetos y visualizarlos

In [2]:
symbolic_dir('objetos/')

In [2]:
mesh = o3d.io.read_triangle_mesh('objetos/' + 'meshes/model.obj', True) #read mesh adding true enables to read the textures

In [39]:
o3d.visualization.draw_geometries([mesh])

In [ ]:
### TODO: Necesitamos crear los links simbolicos para poder acceder a las texturas sin necesidad de modificar la ubicación del archivo

In [42]:
mesh.get_axis_aligned_bounding_box()

AxisAlignedBoundingBox: min: (1.97345, 1.97374, 2), max: (2.02656, 2.02624, 2.08882)

### Extraer imagenes RGB-D

Por comando recibe el modelo 3D (nombre) y los 6 parametros extrinsecos de la cámara. 
Almacena archivos (imágenes) y nube de puntos

In [4]:
def Get_Pointcloud(scene, fov, center, eye, up, width, height, direccion, i):
  # (scene, fov, center, eye, up, width, height)
  rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
    fov_deg= fov,
    center= center,
    eye= eye,
    up= up,
    width_px=width,
    height_px=height,
  )
  # We can directly pass the rays tensor to the cast_rays function.
  ans = scene.cast_rays(rays)
  hit = ans['t_hit'].isfinite()
  points = rays[hit][:,:3] + rays[hit][:,3:]*ans['t_hit'][hit].reshape((-1,1))
  pcd = o3d.t.geometry.PointCloud()
  pcd.point["positions"] = o3d.core.Tensor(points.numpy())
  
  if i == 0:
    p_c = pcd
  else:
    p_c.point["positions"] = o3d.core.concatenate((p_c.point["positions"], pcd.point["positions"]), 0)

  o3d.t.io.write_point_cloud(direccion + "clouds/cloud_{}.pcd".format(i), p_c, write_ascii=True) #almacena nubes de puntos
  

In [43]:
pcd = o3d.io.read_point_cloud('cloud.pcd', format='xyz') #lee nubes de puntos

In [33]:
def Get_RGBD(render, fov, center, eye, up, direccion, i):
   '''render : es el objeto de escena que contiene el objeto
      fov : vertical_field of view
      center: camera center (position)
      eye: camera eye (orientation, where the camera see)
      up: camera up vector ()
      direccion: carpeta donde se almacena 
      i = indice de la imagen
      '''
   render.setup_camera(fov, center, eye, up)
   img = render.render_to_image()
   depth = render.render_to_depth_image()
   o3d.io.write_image(direccion +"/RGB/RGB_{}.png".format(i), img, 9)
   o3d.io.write_image(direccion + "/Depth/D_{}.png".format(i), depth, 9)

In [34]:
### MAIN
global p_c

fov = 17.7064
img_H = 100
img_W = 100
up = [0, 1, 0]
cent = [0.0,0.5,0]
eye = [2.0,1.0,2.0]

#Cargamos malla
mesh = o3d.io.read_triangle_mesh('objetos/' + 'meshes/model.obj', True)

# Raycasting
mesh1 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(mesh1)
# render for RGBD images
render = o3d.visualization.rendering.OffscreenRenderer(width=img_W, height=img_H) #Linux only
render.scene.add_geometry('mesh', mesh)

# RGBD and pointcloud extraction

Get_Pointcloud(scene, fov, cent, eye, up, img_W, img_H, direccion, i)
Get_RGBD(render,  fov, cent, eye, up, direccion, i)


RuntimeError: [Open3D Error] (__cdecl open3d::visualization::rendering::EngineInstance::EngineInstance(void)) D:\a\Open3D\Open3D\cpp\open3d\visualization\rendering\filament\FilamentEngine.cpp:104: EGL Headless is not supported on this platform.


Una alternativa es probar con Application y renderizar la de color y profundidad

Pendiente ver si es posible utilzar esta misma para la nube de puntos.

In [43]:
import os
from math import floor
import shutil

In [83]:
files = os.listdir('MODELDATASET/')
files.sort()
l = files.__len__()
train = int(l *.7)
os.mkdir('incompleto')
os.mkdir('completo')
os.mkdir('incompleto_val')
os.mkdir('completo_val')
i = 1
for f in files:

    if f[11:] == 'In.npy' or f[10:] == 'In.npy' or f[9:] == 'In.npy':
        if i < train:
            new_path = 'incompleto/' + f
            shutil.move('MODELDATASET/'+ f, new_path)
        else:
           new_path = 'incompleto_val/' + f
           shutil.move('MODELDATASET/'+f, new_path)
        
    else:
        if i < train:
            new_path = 'completo/' + f
            shutil.move('MODELDATASET/'+f, new_path)
        else:
           new_path = 'completo_val/' + f
           shutil.move('MODELDATASET/'+f, new_path)
    i += 1

In [82]:
int(l *.7 )

420

In [81]:
i

601